In [320]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Team Casimir Funk Dietary Reference Intakes Notebook
In this notebook, we create the our dietary reference intakes functions.

In [393]:
def handle_query(age, sex):
    '''
    Description
    --------------------------------------------------
    This is a function to handle the Age & Sex Query 
    for our user.
    
    Inputs
    --------------------------------------------------
    + age : value that can be converted to an integer
    + sex : string that is either Male or Female
    
    Outputs
    --------------------------------------------------
    + key : a key that corresponds to a column in
            us_diet_minimums.csv
    + diet_mins : a dataframe that holds the data 
            from us_diet_minimums.csv
    '''
    age = int(age)
    sex = sex.title()
    diet_mins = pd.read_csv('./data/us_diet_minimums.csv')
    diet_mins.columns
    
    if (sex == 'Male' or sex == 'Female') and (age > 0):
        s = sex[0]
        age_cols = diet_mins.columns[2:]
        ages = [ent.split(" ")[1].replace("+", "") for ent in age_cols]
        ages = [ages[i] for i in range(len(ages)) if ages[i - 1] != ages[i]] # Removes duplicates
        ages_min = np.array([int(age.split("-")[0]) for age in ages])

        age_range = ages[sum(ages_min <= age) - 1]
        if age_range == '1-3':
            s = 'C'
        if age_range == '51':
            age_range += '+'

        key = f'{s} {age_range}'
        return key, diet_mins
    else:
        return -29, diet_mins # -29 is error code for invalid inputs

In [395]:
def get_drefs(key, diet_mins):
    '''
    Description
    --------------------------------------------------
    This is a function that returns the Dietary 
    References from us_diet_minimums.csv for the 
    user's desired age and sex
    
    Inputs
    --------------------------------------------------
    + key : a key that corresponds to a column in
            us_diet_minimums.csv
    + diet_mins : a dataframe that holds the data 
            from us_diet_minimums.csv
    
    Outputs
    --------------------------------------------------
    + drefs : the dietary reference intakes from 
            us_diet_minimums.csv for the  user's desired age 
            and sex
    '''
    if key != -29:
        drefs = diet_mins[['Nutrition', key]].set_index('Nutrition')
        return drefs
    else:
        return -29 # -29 is error code for invalid inputs

In [397]:
def dref_wrapper(age, sex):
    '''
    Description
    --------------------------------------------------
    This is a wrapper function that combines the
    functionality of handle_query() & get_drefs()

    Inputs
    --------------------------------------------------
    + age : value that can be converted to an integer
    + sex : string that is either Male or Female
    
    Outputs
    --------------------------------------------------
    + drefs : the dietary reference intakes from 
            us_diet_minimums.csv for the  user's 
            desired age and sex
    '''
    key, diet_mins = handle_query(age, sex)
    drefs = get_drefs(key, diet_mins)
    if type(drefs) != int:
        return drefs
    else:
        print('Enter a Valid Age and Sex')
        return -29 # -29 is error code for invalid inputs

In [399]:
def compare_groups(groups):
    '''
    Description
    --------------------------------------------------
    This is a function that allows the user to compare
    the dietary reference intakes for different 
    groups.

    Inputs
    --------------------------------------------------
    + groups : a list of (age, sex) tuples where age
            is a value that can be converted to 
            an integer and sex is a string that is 
            either Male or Female
    
    Outputs
    --------------------------------------------------
    + comp_df : the dietary reference intakes from 
            us_diet_minimums.csv for the  user's 
            desired groups
    '''
    # Empty dataframe that we will add our groups to compare
    comp_df = pd.DataFrame()
    valid_groups = True
    
    for group in groups:
        drefs = dref_wrapper(group[0], group[1])
        if type(drefs) == int:
            print(f'Invalid Group: Age = {group[0]}; Sex = {group[1]}')
            valid_groups = False
            break
        else:
            col = drefs.columns[0]
            comp_df[col] = drefs[col]
    
    if valid_groups: # Only returns comp_df if all groups valid
        return comp_df
    else:
        return -29 # -29 is error code for invalid inputs

In [401]:
dref_wrapper(age = -2, sex = 'Male')

Enter a Valid Age and Sex


-29